아래의 코드는 주어진 데이터를 분석하여 고장을 감지하는 과정을 수행합니다. 

다음과 같은 단계로 이루어져 있습니다:

> 1. 필요한 라이브러리와 사용자 정의 모듈을 불러옵니다.

> 2. 데이터 파일 경로를 설정합니다.

> 3. 결과를 저장하기 위한 함수를 정의합니다.

> 4. 훈련 데이터와 테스트 데이터를 불러옵니다.

> 5. 훈련 데이터를 사용하여 Configuration을 계산하고 저장합니다.

> 6. 시험용 데이터를 사용하여 Univariate Monitoring을 실행합니다.

> 7. 각 결과를 CSV 파일로 저장합니다.

In [1]:
# 필요한 라이브러리들을 불러옵니다.
import os, sys
import numpy as np
import pandas as pd
from tqdm import tqdm

# 상위 디렉토리로 경로를 설정합니다.
sys.path.append(os.path.abspath('../'))

# 사용자 정의 모듈을 불러옵니다.
from classify_fault.set_config import calculate_variables_config, save_config, load_config
from classify_fault.fault_detection import detect_fault

In [2]:
data_path = '../data/TemperatureTrain.csv'
data_path2 = '../data/Temperature.csv'
config_save_path = '../config/example_config.json'

In [3]:
# 훈련 데이터와 테스트 데이터를 불러옵니다.
train_data = pd.read_csv(data_path, index_col=[0])
test_data = pd.read_csv(data_path2, index_col=[0])
tag_list = train_data.columns.to_list()
test_tag_list = ['temp5Normal', 'temp5Bias', 'temp5Frozen', 'temp5Drift', 'temp5Gain']

In [4]:
train_data[tag_list].head()

,temp5
DateTime,
2022-08-01 00:00:00,1220.54
2022-08-01 00:10:00,1210.31
2022-08-01 00:20:00,1206.14
2022-08-01 00:30:00,1200.39
2022-08-01 00:40:00,1204.22


In [5]:
test_data[test_tag_list].head()

,temp5Normal,temp5Bias,temp5Frozen,temp5Drift,temp5Gain
DateTime,,,,,
2023-01-01 00:00:00,1192.58,1192.58,1192.58,1192.58,1192.58
2023-01-01 00:01:00,1193.04,1193.04,1193.04,1193.04,1193.04
2023-01-01 00:02:00,1193.46,1193.46,1193.46,1193.46,1193.46
2023-01-01 00:03:00,1192.96,1192.96,1192.96,1192.96,1192.96
2023-01-01 00:04:00,1190.98,1190.98,1190.98,1190.98,1190.98


In [6]:
# 훈련 데이터를 사용하여 Configuration을 계산하고 저장합니다.
config = calculate_variables_config(tag_list=tag_list, data=train_data[tag_list].values)
save_config(data=config, json_file_path=config_save_path)

tag = tag_list[0]    # 한 개의 태그 이용

# 시험 태그 설정
test_tag = 'temp5Drift'
test_tag_idx = test_data.columns.get_loc(test_tag)
type_to_check_ = {"frozen": False,
                  "boundary": False,
                  "dynamics": False,
                  "drift": True}

In [7]:
# 시험용 데이터를 사용하여 Univariate Monitoring을 실행합니다.
total_results = []

for i in tqdm(range(10, len(test_data))):
    config = load_config(json_file_path=config_save_path)
    tracking_size = config[tag]["tracking_size"]
    type_to_check = config[tag]["type_to_check"]
    frozen_threshold = config[tag]["frozen_threshold"]
    boundary_limits = config[tag]["boundary_limits"]
    dynamic_threshold = config[tag]["dynamic_threshold"]
    drift_params = config[tag]["drift_params"]        

    test_ = test_data.values[i - tracking_size: i, test_tag_idx]
    result = detect_fault(data=test_, tag=tag,
                            tracking_size=tracking_size, 
                            frozen_threshold=frozen_threshold,
                            boundary_limits=boundary_limits,
                            dynamic_threshold=dynamic_threshold, 
                            drift_params=drift_params,
                            type_to_check=type_to_check_,
                            config_path=config_save_path)
    total_results.append(result)

100%|██████████| 52742/52742 [01:41<00:00, 519.01it/s]


In [8]:
total_results[-1]

{'success': True,
 'fault_detected': True,
 'Frozen': False,
 'Boundary': False,
 'Dynamics': False,
 'Drift': True,
 'message': '-',
 'values': {'frozen': None,
  'boundary': None,
  'dynamics': None,
  'drift': [227628.64304371437, 0]}}